Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST_cln.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (187248, 28, 28) (187248,)
Validation set (8905, 28, 28) (8905,)
Test set (8680, 28, 28) (8680,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (187248, 784) (187248, 10)
Validation set (8905, 784) (8905, 10)
Test set (8680, 784) (8680, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
hidden_units = 1024
IMAGE_PIXELS = image_size * image_size
lmbda = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, IMAGE_PIXELS))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Hidden layer
    with tf.name_scope('hidden'):
        weights1 = tf.Variable(
            tf.truncated_normal([IMAGE_PIXELS, hidden_units],
                                stddev=1.0 / np.sqrt(float(IMAGE_PIXELS))),
            name='weights')
        biases1 = tf.Variable(tf.zeros([hidden_units]),
                              name='biases')
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)

    with tf.name_scope('softmax_linear'):
        weights2 = tf.Variable(
            tf.truncated_normal([hidden_units, num_labels],
                                stddev=1.0 / np.sqrt(float(hidden_units))),
            name='weights')
        biases2 = tf.Variable(tf.zeros([num_labels]),
                              name='biases')
        logits = tf.matmul(hidden, weights2) + biases2

    # Training computation.
    l2_norm_squared = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + lmbda * l2_norm_squared
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2 )
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2 )

In [6]:
num_steps = int(train_dataset.shape[0]/batch_size)
epochs = 5

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for epoch in xrange(epochs):
        print("epoch %d" % epoch)
        for step in xrange(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            batch_mask = np.random.randint(0, train_dataset.shape[0], batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[[batch_mask]]
            batch_labels = train_labels[[batch_mask]]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
                [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
epoch 0
Minibatch loss at step 0: 2.715548
Minibatch accuracy: 14.1%
Validation accuracy: 41.1%
Minibatch loss at step 500: 0.605834
Minibatch accuracy: 90.6%
Validation accuracy: 84.4%
Minibatch loss at step 1000: 0.689301
Minibatch accuracy: 85.9%
Validation accuracy: 86.1%
epoch 1
Minibatch loss at step 0: 0.634604
Minibatch accuracy: 85.2%
Validation accuracy: 86.6%
Minibatch loss at step 500: 0.515175
Minibatch accuracy: 89.8%
Validation accuracy: 87.0%
Minibatch loss at step 1000: 0.502355
Minibatch accuracy: 88.3%
Validation accuracy: 86.9%
epoch 2
Minibatch loss at step 0: 0.509277
Minibatch accuracy: 88.3%
Validation accuracy: 87.7%
Minibatch loss at step 500: 0.615074
Minibatch accuracy: 82.8%
Validation accuracy: 87.6%
Minibatch loss at step 1000: 0.675266
Minibatch accuracy: 83.6%
Validation accuracy: 87.2%
epoch 3
Minibatch loss at step 0: 0.590252
Minibatch accuracy: 84.4%
Validation accuracy: 87.8%
Minibatch loss at step 500: 0.442693
Minibatch accuracy: 89.8

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
batch_size = 128
hidden_units = 1024
IMAGE_PIXELS = image_size * image_size
lmbda = 0.0

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, IMAGE_PIXELS))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Hidden layer
    with tf.name_scope('hidden'):
        weights1 = tf.Variable(
            tf.truncated_normal([IMAGE_PIXELS, hidden_units],
                                stddev=1.0 / np.sqrt(float(IMAGE_PIXELS))),
            name='weights')
        biases1 = tf.Variable(tf.zeros([hidden_units]),
                              name='biases')
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)

    with tf.name_scope('softmax_linear'):
        weights2 = tf.Variable(
            tf.truncated_normal([hidden_units, num_labels],
                                stddev=1.0 / np.sqrt(float(hidden_units))),
            name='weights')
        biases2 = tf.Variable(tf.zeros([num_labels]),
                              name='biases')
        logits = tf.matmul(hidden, weights2) + biases2

    # Training computation.
    l2_norm_squared = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + lmbda * l2_norm_squared
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2 )
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2 )

In [8]:
np.random.seed(0)
rand_choos = np.random.randint(0, train_dataset.shape[0], 20000)
small_train_dataset = train_dataset[[rand_choos]]
small_train_labels = train_labels[[rand_choos]]
num_steps = int(small_train_dataset.shape[0]/batch_size)
epochs = 5

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for epoch in xrange(epochs):
        print("epoch %d" % epoch)
        for step in xrange(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            batch_mask = np.random.randint(0, 20000, batch_size)
            # Generate a minibatch.
            batch_data = small_train_dataset[[batch_mask]]
            batch_labels = small_train_labels[[batch_mask]]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
            _, l, predictions = session.run(
                [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
epoch 0
Minibatch loss at step 0: 2.289205
Minibatch accuracy: 6.2%
Validation accuracy: 34.8%
epoch 1
Minibatch loss at step 0: 0.335780
Minibatch accuracy: 92.2%
Validation accuracy: 83.2%
epoch 2
Minibatch loss at step 0: 0.294948
Minibatch accuracy: 93.0%
Validation accuracy: 84.0%
epoch 3
Minibatch loss at step 0: 0.296443
Minibatch accuracy: 93.0%
Validation accuracy: 84.2%
epoch 4
Minibatch loss at step 0: 0.300687
Minibatch accuracy: 89.1%
Validation accuracy: 83.9%
Test accuracy: 92.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [9]:
batch_size = 128
hidden_units = 1024
IMAGE_PIXELS = image_size * image_size
lmbda = 0.001
drop_ratio = 0.5

# Helper functions
def init_weights(shape):
    return tf.Variable( tf.truncated_normal(shape, stddev=1.0 / np.sqrt(float(shape[0]))) )

def init_biases(shape):
    return tf.Variable(tf.zeros(shape[1]))

def network(layers):
    return [ (init_weights([layers[i], layers[i+1]]), init_biases([layers[i], layers[i+1]]))
           for i in range(len(layers)-1)]

def feedforward(data, net_params, drop_out):
    hid = data
    for w, b in net_params[:-1]:
        hid = tf.nn.relu(tf.matmul(hid, w) + b)
        hid = tf.nn.dropout(hid, drop_out)
    return tf.matmul(hid, net_params[-1][0]) + net_params[-1][1]


# Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, IMAGE_PIXELS))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

# Parameters
net_params = network([IMAGE_PIXELS, hidden_units, num_labels])

# Training computation.
dropout = tf.placeholder("float")
logits = feedforward(tf_train_dataset, net_params, dropout)
l2_norm_squared = sum([tf.nn.l2_loss(w) for w, b in net_params])
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + lmbda * l2_norm_squared
  
# Optimizer.
optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# Predictions for the training, validation, and test data.
dataset = tf.placeholder(tf.float32, shape=[None, IMAGE_PIXELS])
pred = tf.nn.softmax(feedforward(dataset, net_params, dropout))

In [10]:
np.random.seed(0)
rand_choos = np.random.randint(0, train_dataset.shape[0], 20000)
small_train_dataset = train_dataset[[rand_choos]]
small_train_labels = train_labels[[rand_choos]]
num_steps = int(small_train_dataset.shape[0]/batch_size)
epochs = 5

with tf.Session() as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for epoch in xrange(epochs):
        print("epoch %d" % epoch)
        for step in xrange(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            batch_mask = np.random.randint(0, 20000, batch_size)
            # Generate a minibatch.
            batch_data = small_train_dataset[[batch_mask]]
            batch_labels = small_train_labels[[batch_mask]]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            _, l = session.run([optimizer, loss], 
                    feed_dict={tf_train_dataset: batch_data, tf_train_labels: batch_labels, dropout: drop_ratio})
            batch_pred = session.run(pred, feed_dict={dataset: batch_data, dropout: 1})
            if (step % 500 == 0):
                valid_pred = session.run(pred, feed_dict={dataset: valid_dataset, dropout: 1})
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(batch_pred, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                        valid_pred, valid_labels))
    test_pred = session.run(pred, feed_dict={dataset: test_dataset, dropout: 1})
    print("Test accuracy: %.1f%%" % accuracy(test_pred, test_labels))

Initialized
epoch 0
Minibatch loss at step 0: 2.825914
Minibatch accuracy: 45.3%
Validation accuracy: 31.8%
epoch 1
Minibatch loss at step 0: 0.787973
Minibatch accuracy: 94.5%
Validation accuracy: 82.3%
epoch 2
Minibatch loss at step 0: 0.694874
Minibatch accuracy: 94.5%
Validation accuracy: 83.2%
epoch 3
Minibatch loss at step 0: 0.683679
Minibatch accuracy: 96.1%
Validation accuracy: 83.7%
epoch 4
Minibatch loss at step 0: 0.821206
Minibatch accuracy: 94.5%
Validation accuracy: 83.5%
Test accuracy: 91.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [6]:
batch_size = 64
hidden_units = 512
IMAGE_PIXELS = image_size * image_size
lmbda = 0.0001
drop_ratio = [1, 0.5, 0.5, 0.5]

# Helper functions
def init_weights(shape):
    return tf.Variable( tf.truncated_normal(shape, stddev=1.0 / np.sqrt(float(shape[0]))) )

def init_biases(shape):
    return tf.Variable(tf.zeros(shape[1]))

def network(layers, drop_param):
    # to store the hypothesis (structure and dropout)
    return [ (init_weights([layers[i], layers[i+1]]), init_biases([layers[i], layers[i+1]]), drop_param[i])
           for i in range(len(layers)-1)]

def feedforward(data, net_params):
    hid = data
    for w, b, d in net_params[:-1]:
        hid = tf.nn.relu(tf.matmul(hid, w) + b)
        hid = tf.nn.dropout(hid, d)
    return tf.matmul(hid, net_params[-1][0]) + net_params[-1][1]


# Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, IMAGE_PIXELS))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

# Parameters
drop_param = tf.placeholder(tf.float32, shape=(4,))
net_params = network([IMAGE_PIXELS, hidden_units, hidden_units, hidden_units/2, num_labels],
                     drop_param)

# Training computation.
logits = feedforward(tf_train_dataset, net_params)
l2_norm_squared = sum([tf.nn.l2_loss(w) for w, _, _ in net_params])
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + lmbda * l2_norm_squared

# Optimizer.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

# Predictions for the training, validation, and test data.
dataset = tf.placeholder(tf.float32, shape=[None, IMAGE_PIXELS])
pred = tf.nn.softmax(feedforward(dataset, net_params))

In [9]:
from time import time
num_steps = int(train_dataset.shape[0]/batch_size)
epochs = 10

t0 = time()
with tf.Session() as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for epoch in xrange(epochs):
        print("epoch %d" % epoch)
        for step in xrange(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            batch_mask = np.random.choice(np.arange(train_dataset.shape[0]), batch_size, replace=False)
            # Generate a minibatch.
            batch_data = train_dataset[[batch_mask]]
            batch_labels = train_labels[[batch_mask]]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            _, l = session.run([optimizer, loss], 
                    feed_dict={tf_train_dataset: batch_data, tf_train_labels: batch_labels, drop_param: drop_ratio})
            batch_pred = session.run(pred, feed_dict={dataset: batch_data, drop_param: [1,1,1,1]})
            if (step % 500 == 0):
                valid_pred = session.run(pred, feed_dict={dataset: valid_dataset, drop_param: [1,1,1,1]})
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(batch_pred, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_pred, valid_labels))
    print("\nElapsed training time: {0}\n".format(time()-t0))
    test_pred = session.run(pred, feed_dict={dataset: test_dataset, drop_param: [1,1,1,1]})
    print("Test accuracy: %.1f%%" % accuracy(test_pred, test_labels))

Initialized
epoch 0
Minibatch loss at step 0: 2.373595
Minibatch accuracy: 20.3%
Validation accuracy: 15.1%
Minibatch loss at step 500: 0.742103
Minibatch accuracy: 87.5%
Validation accuracy: 82.7%
Minibatch loss at step 1000: 0.607730
Minibatch accuracy: 87.5%
Validation accuracy: 83.6%
Minibatch loss at step 1500: 0.460959
Minibatch accuracy: 95.3%
Validation accuracy: 84.4%
Minibatch loss at step 2000: 0.418627
Minibatch accuracy: 92.2%
Validation accuracy: 85.2%
Minibatch loss at step 2500: 0.388685
Minibatch accuracy: 93.8%
Validation accuracy: 86.1%
epoch 1
Minibatch loss at step 0: 0.637530
Minibatch accuracy: 84.4%
Validation accuracy: 86.6%
Minibatch loss at step 500: 0.594848
Minibatch accuracy: 90.6%
Validation accuracy: 86.5%
Minibatch loss at step 1000: 0.623489
Minibatch accuracy: 95.3%
Validation accuracy: 87.1%
Minibatch loss at step 1500: 0.493466
Minibatch accuracy: 90.6%
Validation accuracy: 87.2%
Minibatch loss at step 2000: 0.414581
Minibatch accuracy: 90.6%
Valida